In [ ]:
import pandas as pd
import os
import numpy as np
import re
import sys
from pandas import DataFrame
from sklearn.utils import shuffle
from collections import OrderedDict
import hashlib

In [ ]:
from log_mining import parse_hdfs_file,read_file, dict_label

In [ ]:
from Extractor import FeatureExtractor

In [ ]:
APACHE_ACCESS_LOG_PATTERN = '^(\S+) (\S+) (\S+) (\S+) (\S+) (.*)'


In [ ]:
fo = open("abc.txt", "r")    

In [ ]:
dta = parse_hdfs_file(fo,APACHE_ACCESS_LOG_PATTERN)

In [ ]:
data = pd.DataFrame(dta,columns=["Date","Time","Pid","Info","Component","Content","EventTemplate","EventId"])

In [ ]:
df_train_append = read_file(data)

In [ ]:
label_csv = pd.read_csv('anomaly_label.csv')

In [ ]:
df_event = dict_label(df_train_append,label_csv)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df_train_append['EventSequence'].values, df_train_append['Label'].values, test_size = 0.2) 

In [ ]:
from collections import Counter
from scipy.special import expit
from itertools import compress

In [ ]:
feature_extractor = FeatureExtractor()
x_train = feature_extractor.fit_transform(x_train, term_weighting='tf-idf')
x_test = feature_extractor.transform(x_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfclf = RandomForestClassifier(max_depth=1, random_state=42)
rfclf = rfclf.fit(x_train, y_train)

from sklearn.metrics import f1_score

predicted_y = rfclf.predict(x_test)
print("test accuracy: ",f1_score(y_test, predicted_y, average='micro'))

In [ ]:
from sklearn.externals import joblib  

joblib.dump(rfclf, "train-model.pkl")